In [1]:
import spdcalc as spdc
import numpy as np
import api_v2 as a2

Set up the downconversion with desired properties, initializing the photons and the crystal separately. 

In [2]:
theta = 0
phi_s = 0
phi_i = np.pi
lambda_s = lambda_i = 1550e-9
lambda_p = 775e-9
waist_spdc_x = 100e-6
waist_spdc_y = 100e-6
waist_spdc = (waist_spdc_x, waist_spdc_y)
waist_pump = waist_spdc
crystal = a2.Crystal(crystal_type = 'KDP_1', pm_type = 'Type1_e_oo', periodic_poling=False)
signal = spdc.Photon.signal(phi_s, theta, lambda_s, waist_spdc)
idler = spdc.Photon.idler(phi_i, theta, lambda_i, waist_spdc)
pump = spdc.Photon.pump(lambda_p, waist_pump)

dcObj = a2.Downconversion(signal, idler, pump, crystal)

Set the dimension and then assign optimum properties to the setup. Also generate the signal and idler wavelength arrays and use these to generate a JSA.

In [3]:
dim = 200

dcObj.rust_spdc.pump_bandwidth = 5.35e-9
dcObj.assign_optimum_theta()

signal_array = idler_array = np.linspace(1450e-9, 1650e-9, dim)
JSA = a2.calc_jsa(dcObj, signal_array, idler_array)

Next, use the numpy linear algebra library to calculate the singular value decomposition of the JSA matrix. This function returns a three-element tuple, the second of which is a one-dimensional array of Schmidt coefficients, $\lambda_i$. To ensure proper normalization, we impose the condition that

$$ \sum\limits_{i}^N \lambda_i^2 = 1. $$

In [4]:
a = np.linalg.svd(JSA)[1]
a_norm = a/np.linalg.norm(a)

Then we can calculate the single-photon purity, equal to 

$$ P = \sum\limits_{i}^N \lambda_i^4 $$

which can then be used to calculate the Schmidt number using 

$$ K = \frac{1}{P}. $$

In [5]:
purity = np.sum(a_norm**4)

schmidt_num = 1/purity
print(schmidt_num)

7.9316882277263945
